## Activity 1: Working with molecular data

![treachery of images](https://upload.wikimedia.org/wikipedia/en/b/b9/MagrittePipe.jpg)

In this activity, we will learn about the different representations of molecules.

### Molecules as a graph

Let's import `rdkit` and set-up a few things to make structures look nice in notebooks

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.rdBase import BlockLogs
from rdkit.DataStructs.cDataStructs import TanimotoSimilarity 
import numpy as np

# turn off rdkit warnings
block = BlockLogs()

# use SVGs and make molecules fill image rather than all be same scale
Draw.IPythonConsole.ipython_useSVG = True
Draw.IPythonConsole.drawOptions.drawMolsSameScale = False

This block allows us to search for molecules by name and get molecule record from pubchem


In [ ]:
import requests

def get_cids(text):
    """
    Search pubchem and return best matching record
    """  
    url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{}/cids/TXT'.format(text)
    response = requests.get(url)
    cids = response.text.split()
    if len(cids) == 0:
        return None
    else:
        return cids
def get_record(cid):
    """
    Get pubchem record for a given cid and returns molecule as rdkit
    """
    url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/record/SDF'.format(cid)
    response = requests.get(url)
    mol = Chem.MolFromMolBlock(response.text)
    return mol

def get_molecule(text, n_results=1):
    """
    Search pubchem and return best matching record
    """  
    cids = get_cids(text)
    if cids is None:
        return None
    else:
        if n_results == 1:
            return get_record(cids[0])
        else:
            return [get_record(cid) for cid in cids[:n_results]]

In [ ]:
get_molecule('buckyball')

In [ ]:
get_molecule('octanol')

The code below shows how we can access atoms and covalent bonds -- which will be useful to make graphs

In [ ]:
mol = get_molecule('toluene')
mol

In [ ]:
for a in mol.GetAtoms():
    print(f'Atom {a.GetIdx()} with atomic number {a.GetAtomicNum()}')
    neighbors = [n.GetIdx() for n in a.GetNeighbors()]
    
    print(f'\tBonded to {" ".join([str(n) for n in neighbors])}')

Can you revise the code above to build an adjacency matrix and node feature vector?

### Molecules as line notation

In [ ]:
mol = get_molecule('methanol')
smiles = Chem.MolToSmiles(mol)
print('SMILES:', smiles)

In [ ]:
import selfies as sf

selfies = sf.encoder(smiles)
print('SELFIES:', selfies)

In [ ]:
inchi = Chem.MolToInchi(mol)
print('InChi:', inchi)

### Molecules as coordinates

In [ ]:
mol = get_molecule('aspirin')
mol

In [ ]:
Chem.AllChem.EmbedMolecule(mol)

In [ ]:
c = mol.GetConformer()
c.GetPositions()

### Comparing molecules

The most common way to compare molecules is Morgan Fingerprints --- also known as Extended Connectivity FingerPrint (ECFP).  These are vectors that indicate presence of specific substructures. Pairwise similarity is then computed with Tanimoto similarity.

In [ ]:
m1 = get_molecule('aspirin')
m2 = get_molecule('benzene')
fp1 =  AllChem.GetMorganFingerprint(m1, 2)
fp2 =  AllChem.GetMorganFingerprint(m2, 2)
dist = TanimotoSimilarity(fp1, fp2)
print('similarity', dist)

### Finding nearby molecules

One of the common strategies in modern methods is to generate a local neighborhood around a specific structure. We can do this using `exmol` via the STONED method.

In [ ]:
import exmol

smiles = Chem.MolToSmiles(m1)
near_smi, near_tanimoto = exmol.run_stoned(smiles)
# sort them by tanimoto
descend_order = np.argsort(near_tanimoto)[::-1]
near_smi = [near_smi[i] for i in descend_order]

We can now draw a few of them to see what they look like.

In [ ]:
near_mol = [Chem.MolFromSmiles(s) for s in near_smi]
Draw.MolsToGridImage(near_mol, molsPerRow=5)